# 图灵机器人

图灵:一位伟大的数学家、逻辑学家，被誉为“计算机科学之父”，“人工智能之父”。

图灵机器人是一家以语义和对话技术为核心的人工智能公司，在人工智能机器人领域拥有数百项核心专利。公司网站：http://www.turingapi.com/。　

图灵机器人开放平台是北京光年无限科技旗下的个性化人工智能机器人开放平台。通过图灵机器人开放平台，软硬件产品开发者可快速为自己的产品接入一款具备个性化身份属性特征、满足不同场景多轮对话及上下文对话的人工智能机器人，实现产品的对话式交互。

本节我们使用百度AI的语音识别、语音合成，加上图灵机器人公司的智能聊天机器人，制作一个真正智能的小派小派。



# 开始我们的实验

(1)图灵机器人使用:

图灵机器人注册方式参考:https://segmentfault.com/a/1190000013900291

(2)使用我们的图灵机器人的前期准备:

获取图灵机器人的api_key.

接口地址:http://openapi.tuling123.com/openapi/api/v2

(3)使用图灵机器人:

import json
import urllib.request

api_url = "http://openapi.tuling123.com/openapi/api/v2"

req = {
    "perception":
    {
        "inputText":
        {
            "text": "替换为给图灵机器人的信息"
        }
    },

    "userInfo": 
    {
        "apiKey": "替换为自己的的api_key",
        "userId": "OnlyUseAlphabet"
    }
}

将字典格式的req编码为utf8
req = json.dumps(req).encode('utf8')

提交自己问题给图灵机器人，图灵机器人的回答在response_dic

    http_post = urllib.request.Request(api_url, data=req, headers={'content-type': 'application/json'})
    response = urllib.request.urlopen(http_post)
    response_str = response.read().decode('utf8')
    response_dic = json.loads(response_str)

图灵机器人返回的文本内容
response_dic['results'][0]['values']['text']

返回参数定义在:https://www.kancloud.cn/turing/www-tuling123-com/718227


In [ ]:
######################################################
#
# 先感受下智能小派,选择本cell，按shirt+enter键运行本模块
#
######################################################

#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
import time
import json
import urllib.request

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_speech_recognition import baidu_speech_2_word
from baidu_sound import baidu_word_2_sound
import voice_record

""" 公开课版本的有限量的语音识别，必须更换为自己的api接口　"""
SP_APP_ID = '20733318 '
SP_API_KEY = 'DZiN4wGf2GE7zRZ3xmuIUaVY'
SP_SECRET_KEY = 'cMgwc303sBz0bQ50yDFiKBH1ovb1vxrw'

#图灵机器人api接口
api_url = "http://openapi.tuling123.com/openapi/api/v2"

#图灵机器人的api key 更换为自己的api key
tuling_key = "cdb5f9d27e54421a990bc9c82c0d9e19"

#与图灵机器人聊天
def tuling_robot_func(speak_msg):
    #填充发给图灵机器人的内容
    req = {
        "perception":
        {
            "inputText":
            {
                "text": speak_msg
            },
        },

        "userInfo": 
        {
            "apiKey": tuling_key,
            "userId": "OnlyUseAlphabet"
        }
    }
        # 将字典格式的req编码为utf8
    req = json.dumps(req).encode('utf8')
    
    #给图灵机器人发送请求
    http_post = urllib.request.Request(api_url, data=req, headers={'content-type': 'application/json'})
    response = urllib.request.urlopen(http_post)
    response_str = response.read().decode('utf8')
    response_dic = json.loads(response_str)
    
    #返回图灵机器人的回答
    return response_dic['results'][0]['values']['text']

if __name__ == "__main__":
    try:
        #传入百度AI的参数给语音识别模块
        sp_2_wd = baidu_speech_2_word(SP_APP_ID, SP_API_KEY, SP_SECRET_KEY)
        
        #传入百度AI的参数给语音合成模块
        word_2_sound = baidu_word_2_sound(SP_APP_ID, SP_API_KEY, SP_SECRET_KEY)
        
        while True:
            #进行麦克风录音，每次录音6秒
            voice_record.recording("repeater_raw.wav", 6)

            #将音频采样率更改为百度需要的16000
            os.system('sox repeater_raw.wav -r 16000 repeater.wav')
            
            #对音频文件进行分析
            msg = sp_2_wd.speech_2_word('repeater.wav')

            
            #语音识别正确，则进行语音合成和输出
            if msg["err_no"] == 0:
                #输出语音识别后，转化的文字
                print(msg["result"][0])
        
                #与图灵机器人进行聊天
                tuling_answer = tuling_robot_func(msg["result"][0])
                
                #输出图灵机器人给出的答案
                print(tuling_answer)

                #进行语音合成
                word_2_sound.trans_word_to_sound(tuling_answer,'repeater.wav')
                
                #播放合成的语音
                os.system('mplayer ' + 'repeater.wav')

        #删除临时文件
        os.remove('repeater.wav')
        os.remove('repeater_raw.wav')
        
    except KeyboardInterrupt:
        #删除临时文件
        os.remove('repeater.wav')
        os.remove('repeater_raw.wav')
        print("任务被终止了")

# 让我们临摹代码，学习“语音识别”功能

In [ ]:
#导入标准库
# import sys

# import os

# import time

# import json

# import urllib.request


# #导入自定义库
# sys.path.append('../baidu_api_lib')

# from baidu_speech_recognition import baidu_speech_2_word

# from baidu_sound import baidu_word_2_sound

# import voice_record


# """ 公开课版本的有限量的语音识别，必须更换为自己的api接口　"""
# SP_APP_ID = '20733318 '

# SP_API_KEY = 'DZiN4wGf2GE7zRZ3xmuIUaVY'

# SP_SECRET_KEY = 'cMgwc303sBz0bQ50yDFiKBH1ovb1vxrw'


# #图灵机器人api接口
# api_url = "http://openapi.tuling123.com/openapi/api/v2"


#图灵机器人的api key 更换为自己的api key
# tuling_key = "cdb5f9d27e54421a990bc9c82c0d9e19"


# #与图灵机器人聊天
# def tuling_robot_func(speak_msg):

#     #填充发给图灵机器人的内容
#     req = {

#         "perception":

#         {

#             "inputText":

#             {
#                 "text": speak_msg

#             },

#         },


#         "userInfo": 

#         {
#             "apiKey": tuling_key,

#             "userId": "OnlyUseAlphabet"

#         }

#     }

#         # 将字典格式的req编码为utf8
#     req = json.dumps(req).encode('utf8')

    
#     #给图灵机器人发送请求
#     http_post = urllib.request.Request(api_url, data=req, headers={'content-type': 'application/json'})

#     response = urllib.request.urlopen(http_post)

#     response_str = response.read().decode('utf8')

#     response_dic = json.loads(response_str)

    
#     #返回图灵机器人的回答
#     return response_dic['results'][0]['values']['text']


# if __name__ == "__main__":

#     try:

#         #传入百度AI的参数给语音识别模块
#         sp_2_wd = baidu_speech_2_word(SP_APP_ID, SP_API_KEY, SP_SECRET_KEY)

        
#         #传入百度AI的参数给语音合成模块
#         word_2_sound = baidu_word_2_sound(SP_APP_ID, SP_API_KEY, SP_SECRET_KEY)

        
#         while True:

#             #进行麦克风录音，每次录音6秒
#             voice_record.recording("repeater_raw.wav", 6)


#             #将音频采样率更改为百度需要的16000
#             os.system('sox repeater_raw.wav -r 16000 repeater.wav')

            
#             #对音频文件进行分析
#             msg = sp_2_wd.speech_2_word('repeater.wav')

            
#             #语音识别正确，则进行语音合成和输出
#             if msg["err_no"] == 0:

#                 #输出语音识别后，转化的文字
#                 print(msg["result"][0])

        
#                 #与图灵机器人进行聊天
#                 tuling_answer = tuling_robot_func(msg["result"][0])

                
#                 #输出图灵机器人给出的答案
#                 print(tuling_answer)


#                 #进行语音合成
#                 word_2_sound.trans_word_to_sound(tuling_answer,'repeater.wav')

                
#                 #播放合成的语音
#                 os.system('mplayer ' + 'repeater.wav')


#         #删除临时文件
#         os.remove('repeater.wav')

#         os.remove('repeater_raw.wav')

        
#     except KeyboardInterrupt:

#         #删除临时文件
#         os.remove('repeater.wav')

#         os.remove('repeater_raw.wav')

#         print("任务被终止了")


# 课后练习

(1)设置图灵计算机的机器人设置，配置适合自己的图灵机器人。

(2)图机器人可以进行聊天、脑筋急转弯等对话，尝试聊聊呗